In [1]:
import sys

sys.path.append("..")
from common_utils import set_data_home

set_data_home("~/datasets")
from common_utils import DATA_HOME, join
from lstm.sales_data import Sales_Dataset
import torch
import random

device = "cuda" if torch.cuda.is_available() else "cpu"
SALE_HOME = join(DATA_HOME, "sales_data")
MODEL_HOME = join(DATA_HOME, "sale_model")

torch.manual_seed(42)
random.seed(42)

saved_iter = 500
I, H, B = 528, 528, 4
TRANSFORMER_LAYER = 4
LSTM_LAYER = 2
HEAD = 12
SEQ_LEN = 116
INFER_DAYS = 16

In [2]:
from torch.nn import LSTM, Transformer
from torch import optim
import matplotlib.pyplot as plt
from IPython.display import clear_output
from predictor import Predictor

model = Predictor(I, H, LSTM_LAYER, TRANSFORMER_LAYER, HEAD).cuda()

In [3]:
sd = Sales_Dataset(SALE_HOME, seq_len=SEQ_LEN, is_train=False)

In [4]:
print(sd[1][0].shape, sd[1][1].shape, sd[1][2].shape, sd[1][3])

torch.Size([116, 528]) torch.Size([116, 528]) torch.Size([116, 33]) 2


### Perform Inference

In [5]:
import pandas as pd
from datetime import timedelta

model.load_state_dict(torch.load(f"sales_model_{saved_iter}_{B}.pth"))

sales = pd.read_csv(join(SALE_HOME, "test.csv"), index_col=0)
sales["sales"] = 0.0
# base_sales = sd.base_sales.set_index(["store_nbr", "date"])

for X1, X2, base_sales, store_id in sd:
    X1 = X1.cuda().unsqueeze(0)
    X2 = X2.cuda().unsqueeze(0)
    base_sales = base_sales[[-1]].cuda().unsqueeze(0)
    yhat1, yhat2 = model(X1, X2)

    # infer and update input for each store
    yhat_sales = Sales_Dataset.batched_ret_2_sales(
        base_sales, yhat1[:, [-1], :], yhat2[:, [-1], :]
    )
    yhat_sales = yhat_sales.squeeze(0, 1)
    for i in range(INFER_DAYS):
        # compute actual sales
        ts = (sd.train_max_date + timedelta(days=i + 1)).strftime("%Y-%m-%d")

        # write to each family in the answer dataframe
        for j, f in enumerate(sd.families):
            sales.loc[
                (sales.date == ts)
                & (sales.store_nbr == store_id)
                & (sales.family == f),
                "sales",
            ] = (
                yhat_sales[i * len(sd.families) + j].cpu().item()
            )

### output the answer

In [6]:
sales

,date,store_nbr,family,onpromotion,sales
id,,,,,
3000888,2017-08-16,1,AUTOMOTIVE,0,3.027291
3000889,2017-08-16,1,BABY CARE,0,0.000000
3000890,2017-08-16,1,BEAUTY,2,3.182316
3000891,2017-08-16,1,BEVERAGES,20,2111.487549
3000892,2017-08-16,1,BOOKS,0,0.000000
...,...,...,...,...,...
3029395,2017-08-31,9,POULTRY,1,442.837738
3029396,2017-08-31,9,PREPARED FOODS,0,123.324501
3029397,2017-08-31,9,PRODUCE,1,1100.660034


In [7]:
sales.drop(columns=["store_nbr", "date", "family", "onpromotion"]).sort_values(
    "id"
).sort_index().to_csv("answer.csv", index=True)